# Topic 2: Cloud Optimized Data & STAC

---

In this example we will access the NASA HLS assets, which are archived in cloud optimize geoTIFF (COG) format in the LP DAAC Cumulus cloud space. The COGs can be used like any other geoTIFF file, but have some added features that make them more efficient within the cloud data access paradigm. These features include: overviews and internal tiling. Below we will demonstrate how to leverage these features.

## Import Required Packages

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from collections import defaultdict    # https://stackoverflow.com/questions/26367812/appending-to-list-in-python-dictionary
import requests
import rasterio as rio                 # https://rasterio.readthedocs.io/en/latest/
from rasterio.plot import show
import rioxarray                       # https://corteva.github.io/rioxarray/stable/index.html
import geopandas
import pyproj
from pyproj import Proj
from shapely.ops import transform
import geoviews as gv
from cartopy import crs
import hvplot.xarray
import holoviews as hv
gv.extension('bokeh', 'matplotlib')

---

## Set GDAL Configuration Options

**First, let us set the gdal configuration options for this session**

In [ ]:
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
                   AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))


os.environ.update(env)

## Working with Overviews 

**Access a single HLS asset to identify the overview levels**

In [ ]:
foa_url = "https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TGF.2020191T172901.v1.5.B04.tif"

In [ ]:
with rio.open(foa_url) as src:
    hls_ov_levels = src.overviews(1)

In [ ]:
hls_ov_levels

**Request the second overview level from the asset (`overview_level=1`)**

In [ ]:
%time
with rioxarray.open_rasterio(foa_url, masked=True, overview_level=1, chunks=True) as src:    # https://nbviewer.jupyter.org/gist/rsignell-usgs/f4dd62ad1274c5b5ed69e5a6b81c1295 & http://rasterio.readthedocs.io/en/latest/topics/resampling.html
        fig, ax = plt.subplots(1, figsize=(12, 12))
        print(src)
        show(src, cmap='YlGnBu')

**Request the last overview level (`overview_level=3`)**

In [ ]:
%time
with rioxarray.open_rasterio(foa_url, masked=True, overview_level=3, chunks=True) as src:
    fig, ax = plt.subplots(1, figsize=(12, 12))
    print(src)
    show(src, cmap='YlGnBu')

## Requesting Spatial Subsets

![COG tiling example](img/COG_Smile_AOI.png)

**Now, we will read in a geojson file and use its bounding box to clip the cloud asset in later steps**

In [ ]:
os.listdir("./data/")

In [ ]:
field = geopandas.read_file('./data/ne_w_agfields.geojson')
type(field)

In [ ]:
field

In [ ]:
fieldShape = field['geometry'][0]    # Define the geometry as a shapely polygon
fieldShape

Get the lower-left and upper-right coordinates

In [ ]:
fieldShape.bounds

Use geoviews to combine a basemap with the shapely polygon of our Region of Interest (ROI)

In [ ]:
base = gv.tile_sources.EsriImagery.opts(width=650, height=500)
farmField = gv.Polygons(fieldShape).opts(line_color='yellow', line_width=10, color=None)
base * farmField

### Requests an area of interest

**Transform coordinates from lat lon (units = dd) to UTM (units = m)**

In [ ]:
with rio.open(foa_url) as src:
    hls_proj = src.crs.to_string()

In [ ]:
geo_CRS = Proj('+proj=longlat +datum=WGS84 +no_defs', preserve_units=True)   # Source coordinate system of the ROI
project = pyproj.Transformer.from_proj(geo_CRS, hls_proj)                    # Set up the transformation
fsUTM = transform(project.transform, fieldShape)                             # Transform

**Print the transformed bounds (now in meters)**

In [ ]:
fsUTM.bounds

**Use fsUTM to subset the source HLS tile**

**Requests data at full extent**

In [ ]:
rds = rioxarray.open_rasterio(foa_url, masked=True, chunks=True)

In [ ]:
rds

**Print the spatial reference attribute**

In [ ]:
#rds.spatial_ref
rds.spatial_ref.attrs

**Plot data at full extent**

In [ ]:
rds[0].hvplot.image(x = 'x', y = 'y', crs = hls_proj, rasterize=True, cmap='Reds', width=800, height=600, colorbar=True, tiles = 'ESRI')

**Request data that intersects with the input geoJSON boundary only**

In [ ]:
rds_clipped = rioxarray.open_rasterio(foa_url, masked=True).rio.clip([fsUTM])    # Note: fsUTM must be in a list
rds_clipped

In [ ]:
rds_clipped[0].hvplot.image(x = 'x', y = 'y', crs = hls_proj, rasterize=True, cmap='Reds', width=800, height=600, colorbar=True, tiles = 'ESRI')

**Add the field boudary to the clipped image**

In [ ]:
rds_clipped[0].hvplot.image(x = 'x', y = 'y', crs = hls_proj, rasterize=True, cmap='Reds', width=800, height=600, colorbar=True, tiles = 'ESRI') * farmField

**Get the Geotransformation information for the full tile**

In [ ]:
rds.spatial_ref.GeoTransform

**Geotransformation information for the clipped image**

In [ ]:
rds_clipped.spatial_ref.GeoTransform

### Request data for a point of interest

In [ ]:
from pyproj import Transformer

# convert coordinate to raster projection
lon = -101.66786
lat = 41.05679

In [ ]:
transformer = Transformer.from_crs("EPSG:4326", rioxarray.open_rasterio(foa_url, masked=True).rio.crs, always_xy=True)
xx, yy = transformer.transform(lon, lat)
print(f'X,Y in source units: {xx},{yy}')

In [ ]:
# get value from grid
value = rds.sel(x=xx, y=yy, method="nearest").values
value

## Resources

- https://nbviewer.jupyter.org/gist/rsignell-usgs/f4dd62ad1274c5b5ed69e5a6b81c1295  
- http://rasterio.readthedocs.io/en/latest/topics/resampling.html  
- https://gis.stackexchange.com/questions/358036/extracting-data-from-a-raster/358058#358058

---

## STAC...Because it's that cool!

NASA's CMR STAC does not allow for querries accross the entire NASA catalog. Users must execute searches within provider catalogs (e.g., LPCLOUD) to find the STAC Items they are searching for. All the providers can be found here: <https://cmr.earthdata.nasa.gov/stac/>. In this exercise, we will query the **LPCLOUD** provider to identify STAC Items from the Harmonized Landsat Sentinel-2 (HLS) collection that fall within our area of interest and within our specified time range.

In [ ]:
cmr_stac_search = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/search'  

**Specify the search criteria we are interested in**

In [ ]:
params = {
    'limit': 100,
    'bbox': '-101.67271614074707,41.04754380304359,-101.65344715118408,41.06213891056728',
    'datetime': '2020-01-01T00:00:00Z/2021-01-01T23:59:59Z',
    'collections': ['HLSS30.v1.5', 'HLSL30.v1.5']
}

In [ ]:
hls_items = requests.post(cmr_stac_search, json=params).json()['features']

**Execute a POST request to query the HLS collection**

In [ ]:
# Search for the HLSS30 and HLSL30 items of interest:
hls_items = requests.post(cmr_stac_search, json=params).json()['features']  # Send POST request with S30 and L30 collections included
print(f"The search query returns {len(hls_items)} items")

**Print a single STAC Item**

In [ ]:
hls_items[0]

**Create of list of links for the desired bands**

In [ ]:
evi_band_links = []

In [ ]:
for i in hls_items:
    if i['collection'] == 'HLSS30.v1.5':
        evi_bands = ['B8A', 'B04', 'B02', 'Fmask'] # NIR RED BLUE Quality for S30
    else:
        evi_bands = ['B05', 'B04', 'B02', 'Fmask'] # NIR RED BLUE Quality for L30
        
    for a in i['assets']:
        if any(b==a for b in evi_bands):
            evi_band_links.append(i['assets'][a]['href'])

In [ ]:
len(evi_band_links)

We can see in the list of links that we have multiple tiles that intersect with our region of interest. Since these represent neighboring UTM zones, we will split them the list of links into seperate lists for each tile.

In [ ]:
tile_dicts = defaultdict(list)

In [ ]:
for l in evi_band_links:
    tile = l.split('.')[-6]
    tile_dicts[tile].append(l)

In [ ]:
tile_dicts.keys()

**Create a list of links for each tile**

In [ ]:
tile_links_T14TKL = tile_dicts['T14TKL']
tile_links_T13TGF = tile_dicts['T13TGF']

In [ ]:
len(tile_links_T13TGF)

In [ ]:
tile_links_T13TGF[:10]

**Split the links by band**

In [ ]:
bands_dicts = defaultdict(list)

In [ ]:
for b in tile_links_T13TGF:
    band = b.split('.')[-2]
    bands_dicts[band].append(b)

In [ ]:
bands_dicts.keys()

In [ ]:
len(bands_dicts['B04'])

In [ ]:
bands_dicts['B04'][:10]

## Resources

- https://stackoverflow.com/questions/46899337/convert-raster-time-series-of-multiple-geotiff-images-to-netcdf
- https://medium.com/@bonnefond.virginie/handling-multi-temporal-satellite-images-with-xarray-30d142d3391
- https://docs.dea.ga.gov.au/notebooks/Frequently_used_code/Opening_GeoTIFFs_NetCDFs.html

---

# [Next: Topic 3 - Data Proximate Compute](Topic_3__Data_Proximate_Compute.ipynb)